## 4.3 模型参数的延后初始化

### 4.3.1 延后初始化

由于隐藏层输入个数未知，系统也无法得知该层权重参数的形状。

In [1]:
from mxnet import init, nd
from mxnet.gluon import nn

class MyInit(init.Initializer):
    def _init_weight(self, name, data):
        print('Init', name, data.shape)
        # 实际的初始化逻辑在此省略了
        
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'),
        nn.Dense(10))

net.initialize(init=MyInit())

由此可见，调用initialize函数时并没有真正初始化参数。

In [2]:
X = nd.random_uniform(shape=(2, 20))
Y = net(X)

Init dense0_weight (256, 20)
Init dense1_weight (10, 256)


在根据输入X做前向计算时，系统能够根据输入的形状自动推断出所有层的权重参数的形状。系统在创建这些参数之后，调用MyInit实例对他们进行初始化，然后才进行前向计算。

当然，这个初始化只会在第一次前向计算时被调用。之后我们再运行前向计算net(X)时则不会重新初始化，因此不会再次产生MyInit实例的输出。

In [5]:
Y = net(X)

系统将真正的参数初始化延后到获得足够信息时才执行的行为叫作延后初始化(deferred initialization)。它可以让模型的创建更加简单：只需要定义每个层的输出大小，而不用人工推测他们的输入个数。

但是，在第一次前向计算之前，我们无法直接操作模型参数，例如无法使用data函数和set_data函数来获取和修改参数。因此，我们经常会额外做一次前向计算来迫使参数被真正地初始化。

### 4.3.2 避免延后初始化

如果系统在调用initialize函数时能够知道所有参数的形状，那么延后初始化就不会发生。

- 第一种情况是我们对已初始化的模型重新初始化时。因为参数形状不会发生变化，所以系统能够立即进行重新初始化。

In [6]:
net.initialize(init=MyInit(), force_reinit=True)

Init dense0_weight (256, 20)
Init dense1_weight (10, 256)


- 第二种情况是我们在创建层的时候指定它的输入个数，使系统不需要额外的信息来推测参数形状。

In [8]:
net = nn.Sequential()
net.add(nn.Dense(256, in_units=20, activation='relu'))
net.add(nn.Dense(10, in_units=256))

net.initialize(init=MyInit())

Init dense4_weight (256, 20)
Init dense5_weight (10, 256)


### 小结

- 系统将真正的参数初始化延后到获得足够信息时才执行的行为叫作延后初始化。
- 延后初始化的主要好处是让模型构造更加简单。例如，我们无需人工推测每个层的输入个数。
- 也可以避免延后初始化。

### 练习

如果在下一次前向计算net(X)前改变输入X的形状，包括批量大小和输入个数，会发生什么？

In [9]:
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'),
        nn.Dense(10))

net.initialize(init=MyInit())

X = nd.random_uniform(shape=(2, 20))
Y = net(X)

Init dense6_weight (256, 20)
Init dense7_weight (10, 256)


In [11]:
# 下一次前向计算
X = nd.random_uniform(shape=(4, 60))
Y = net(X)

MXNetError: Shape inconsistent, Provided = [256,20], inferred shape=(256,60)

会报错，显示形状不一致。